In [22]:
import numpy as np
import pymongo
import pandas as pd
from scipy.stats import ks_2samp, describe, skew
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm
import copy
import seaborn as sns
from scipy import stats
from ast import literal_eval
import random

In [30]:
#read in timit_master
gw_dir = '/home/logan/drive/Research/guesswho18/'     #hive
#gw_dir = '/User/logan/Gogle_Drive/Research/guesswho18/'     #iMac

df_timit = pd.read_csv(gw_dir + 'data/timit_master.csv', sep=',',
       dtype = {
           'start_word' : np.int,
           'end_word': np.int,
           'word': np.str,
           'sample_id': np.str,
           'speaker_id': np.str,
           'start_phoneme': np.int,
           'end_phoneme': np.int,
           'arpabet': np.str,
           'ipa': str,
           'filename': np.str,
           'index_phoneme': np.int
       })
print("Timit done...")

#load mongodb
#connect to mongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["exploration"]
table = db['uniform_timit_train']
run_name = 'uniform'

#move mongo db over to pandas dataframe
cursor = table.find()
df = pd.DataFrame(list(cursor))
df = df.drop(columns='_id')
print("DB loaded...")

Timit done...
DB loaded...


In [37]:
#limit speakers to those in mongodb
df_timit = df_timit[df_timit.filepath.isin(df.filepath.unique())]

grouped = df_timit.groupby('filepath')
possible_fakes = []
for key, group in grouped:
    #check if group has more than 10 words
    if group[['start_word', 'word']].drop_duplicates().count()[0] >= 10:
        #add key to my possible fake files
        possible_fakes.append(key)

In [38]:
random.seed(1593)
with open('fake_generation_dataset.txt', 'w') as f:
    for row in random.sample(possible_fakes, 100):
        f.write(row)
        f.write('\n')

array(['FSMA0', 'MKLS0', 'MRWS0', 'FCYL0', 'FDNC0', 'FMJB0', 'FMKF0',
       'MJEB0', 'MKAH0', 'MMDS0', 'MRGS0', 'MRJH0', 'FALK0', 'FDJH0',
       'FSKC0', 'MDBB1', 'MDEF0', 'MDLH0', 'MDNS0', 'MGAF0', 'MHJB0',
       'MLNS0', 'MRJB1', 'FCAG0', 'FJWB1', 'MJEE0', 'MJLS0', 'MTRC0',
       'MTRT0', 'FEXM0', 'FGDP0', 'MDSJ0', 'MGSH0', 'MJDM0', 'MJRG0',
       'MMVP0', 'MPMB0', 'MRML0', 'MTDP0', 'FBCH0', 'FJDM2', 'MSDS0',
       'FCRZ0', 'MBOM0', 'MDLR1', 'MGAK0', 'MRMG0', 'MVRW0', 'MMPM0'],
      dtype=object)